In [13]:
import numpy as np
import pandas as pd
import pydicom as dicom
import matplotlib.pylab as plt
import glob
import SimpleITK as sitk
import re

import os
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
dicom.config.convert_wrong_length_to_UN = True

In [19]:
#source_path = "F:/U-CAN-Lymfom_A/Ashish_SampleResampling/sample_data/"

#source_path = "F:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB"

source_path = "/media/andres/T7 Shield/ucan_lymfom"
# source_path = "F:/ucan_lymfom"

destination_path = os.path.join(source_path, "Excel_files/metadata.xlsx")
selection_dataframe = os.path.join(source_path, "Excel_files/06_11_2023/Final_Selected_exams_from_U-CAN-Lymfom.xlsx" )

In [20]:
destination_path

'/media/andres/T7 Shield/ucan_lymfom/Excel_files/metadata.xlsx'

In [10]:
dataset = pd.read_excel(os.path.join(source_path, selection_dataframe))

In [ ]:
directory_list = list()
for root, dirs, files in os.walk(source_path, topdown=False):
    for name in dirs:
        directory_list.append(os.path.join(root, name))

In [ ]:
remove_list = [ 'PR----BONE-PULM-mm',
                'PR----Lunga-0.6-ax-mm',
                'PR----WB-Venfas-0.6-ax-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----WB-Ben-lunga-0.6-ax-mm',
                'PR----WB-Venfas-3-ax-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----BONE-1.25-AX-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----Lunga-0.6-ax-mm',
                'PR----SAVED-IMAGES-PR-mm',
                'PR----e1-QCFX-S-400-Static-mm',
                'PR----WB-Venfas-0.6-ax-mm',
                'PR----WB-VEN-AX-mm',
                'PR----WB-Ben-lunga-0.6-ax-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----THORAX-AX-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----THORAX-INANDAD-mm',
                'PR----KEY_IMAGES-PR-mm',
                'PR----SAVED-PR-mm',
                'Examinations that miss either CT or PET or both',
                'MR-',
                'sag',
                'cor',
                'ot-'
            ]

keep_list = ["CT-", "PT-"]

In [ ]:
findir_lst = []
rejection_lst = []

for dir in directory_list:
    dir = dir.replace('\\', '/')
    if any(item.lower() in dir.lower() for item in keep_list) and all(
            item.lower() not in dir.lower() for item in remove_list):
        print(dir)
        findir_lst.append(dir)
    else:
        rejection_lst.append(dir)

In [ ]:
len(rejection_lst)

In [11]:
dataset_list = dataset.values.tolist()

In [12]:
ucan_md_dict = {'dir': [],
                'source_dir': [],
                'patient_dir': [],
                'image_dir': [],
                'dicom_img': [],
                'patient_id': [],
                'patient_age': [],
                'patient_sex': [],
                'patient_weight': [],
                'patient_size': [],
                'rows': [],
                'columns': [],
                'num_slices': [],
                'pixel_spacing': [],
                'slice_thickness': [],
                'img_pos': [],
                'img_orient': [],
                'for_uid': [],
                'att_corr': [],
                'recons_method': [],
                'image_type': [],
                'aquisition_dt': [],
                'aquisition_time': [],
                'study_desc': [],
                'series_desc': [],
                'protocol': [],
                'corr_img': [],
                'modality': [],
                'manufacturer': [],
                'manufacturer_model': [],
                'radiopharmaceutical': [],
                'radiopharmaceutical_volume': [],
                'radiopharmaceutical_start_time': [],
                'radionuclide_total_dose': [],
                'radionuclide_half_life': [],
                'radionuclide_positron_fraction': [],
                'radiopharmaceutical_start_date_time': [],
                }

#emptydir_lst = []

for dir in dataset_list:
    #if len(os.listdir(dir))<100:
    #    emptydir_lst.append(dir)
    #    continue

    ucan_md_dict['dir'].append(dir)
    ucan_md_dict['dicom_img'].append(os.listdir(dir)[0])
    
    ucan_md_dict['source_dir'].append(dir.rsplit(sep='/', maxsplit=2)[-3])
    ucan_md_dict['patient_dir'].append(dir.rsplit(sep='/', maxsplit=2)[-2])
    ucan_md_dict['image_dir'].append(dir.rsplit(sep='/', maxsplit=2)[-1])  
                                            
    #print(os.listdir(dir)[0])
    ds = dicom.dcmread(dir + '/' + str(os.listdir(dir)[0]))

    meta_info = str(ds.fix_meta_info)#.split('\n')
    
    max_range = 500
    
    #PatientID
    try:
        id = re.search(r"\b(0010, 0020)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_id'].append(element)
    except:
        ucan_md_dict['patient_id'].append('')
    
    #PatientAge 
    try:
        id = re.search(r"\b(0010, 1010)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_age'].append(element)
    except:
        ucan_md_dict['patient_age'].append('')

    #PatientSex
    try:
        id = re.search(r"\b(0010, 0040)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_sex'].append(element)
    except:
        ucan_md_dict['patient_sex'].append('')

    #Patient Size
    try:
        id = re.search(r"\b(0010, 1020)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_size'].append(element)
    except:
        ucan_md_dict['patient_size'].append('')

    #Patient Weight
    try:
        id = re.search(r"\b(0010, 1030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_weight'].append(element)
    except:
        ucan_md_dict['patient_weight'].append('')
    
    #Rows
    try:
        id = re.search(r"\b(0028, 0010)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['rows'].append(element)
    except:
        ucan_md_dict['rows'].append('')

    #Columns
    try:
        id = re.search(r"\b(0028, 0011)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['columns'].append(element)
    except:
        ucan_md_dict['columns'].append('')

    #Num of slices
    try:
        id = re.search(r"\b(07a1, 1002)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['num_slices'].append(element)
    except:
        ucan_md_dict['num_slices'].append('')

    #Pixel spacing
    try:
        id = re.search(r"\b(0028, 0030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['pixel_spacing'].append(element)
    except:
        ucan_md_dict['pixel_spacing'].append('')

    #Slice thickness
    try:
        id = re.search(r"\b(0018, 0050)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['slice_thickness'].append(element)
    except:
        ucan_md_dict['slice_thickness'].append('')

    #Image positioning
    try:
        id = re.search(r"\b(0020, 0032)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['img_pos'].append(element)
    except:
        ucan_md_dict['img_pos'].append('')

    #Image orientation
    try:
        id = re.search(r"\b(0020, 0037)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['img_orient'].append(element)
    except:
        ucan_md_dict['img_orient'].append('')

    #Frame of reference UID
    try:
        id = re.search(r"\b(0020, 0052)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['for_uid'].append(element)
    except:
        ucan_md_dict['for_uid'].append('')

    #Attenuation Correction Method
    try:
        id = re.search(r"\b(0054, 1101)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['att_corr'].append(element)
    except:
        ucan_md_dict['att_corr'].append('')

    #Reconstruction Method
    try:
        id = re.search(r"\b(0054, 1103)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['recons_method'].append(element)
    except:
        ucan_md_dict['recons_method'].append('')

    #Image Type
    try:
        id = re.search(r"\b(0008, 0008)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['image_type'].append(element)
    except:
        ucan_md_dict['image_type'].append('')

    #Aquisition Date
    try:
        id = re.search(r"\b(0008, 0022)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['aquisition_dt'].append(element)
    except:
        ucan_md_dict['aquisition_dt'].append('')
    
    #Aquisition Time
    try:
        id = re.search(r"\b(0008, 0032)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['aquisition_time'].append(element)
    except:
        ucan_md_dict['aquisition_time'].append('')

    #Study Description
    try:
        id = re.search(r"\b(0008, 1030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['study_desc'].append(element)
    except:
        ucan_md_dict['study_desc'].append('')

    #Series Description
    try:
        id = re.search(r"\b(0008, 103e)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['series_desc'].append(element)
    except:
        ucan_md_dict['series_desc'].append('')

    #Protocol Name 
    try:
        id = re.search(r"\b(0018, 1030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['protocol'].append(element)
    except:
        ucan_md_dict['protocol'].append('')

    #Corrected Image
    try:
        id = re.search(r"\b(0028, 0051)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['corr_img'].append(element)
    except:
        ucan_md_dict['corr_img'].append('')

    #Modality
    try:
        id = re.search(r"\b(0008, 0060)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['modality'].append(element)
    except:
        ucan_md_dict['modality'].append('')

    #Manufacturer
    try:
        id = re.search(r"\b(0008, 0070)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['manufacturer'].append(element)
    except:
        ucan_md_dict['manufacturer'].append('')

    #Manufacturer_model
    try:
        id = re.search(r"\b(0008, 1090)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['manufacturer_model'].append(element)
    except:
        ucan_md_dict['manufacturer_model'].append('')
    # Radiopharmaceutical
    try:
        id = re.search(r"\b(0018, 0031)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical'].append('')
     # Radiopharmaceutical Volume
    try:
        id = re.search(r"\b(0018, 1071)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical_volume'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical_volume'].append('')
     # Radiopharmaceutical Start Time
    try:
        id = re.search(r"\b(0018, 1072)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical_start_time'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical_start_time'].append('')
     # Radionuclide Total Dose
    try:
        id = re.search(r"\b(0018, 1074)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radionuclide_total_dose'].append(element)
    except:
        ucan_md_dict['radionuclide_total_dose'].append('')
     # Radionuclide Half Life
    try:
        id = re.search(r"\b(0018, 1075)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radionuclide_half_life'].append(element)
    except:
        ucan_md_dict['radionuclide_half_life'].append('')
     # Radionuclide Positron Fraction
    try:
        id = re.search(r"\b(0018, 1076)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radionuclide_positron_fraction'].append(element)
    except:
        ucan_md_dict['radionuclide_positron_fraction'].append('')
     # Radiopharmaceutical Start Date Time
    try:
        id = re.search(r"\b(0018, 1078)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical_start_date_time'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical_start_date_time'].append('')


TypeError: listdir: path should be string, bytes, os.PathLike, integer or None, not list

In [ ]:
"""
with open(r'F:/U-CAN-Lymfom_A/Ashish_SampleResampling/metadata_extraction/dir_lessthan_100images.txt', 'w') as fp:
    for item in emptydir_lst:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

len(emptydir_lst)

Done
2399
"""

In [ ]:
ucan_md = pd.DataFrame(ucan_md_dict)
print(ucan_md.shape)
#(8325, 29)
#(8275, 29)
ucan_md.head(2)

In [ ]:
ucan_md.dtypes
ucan_md['imgsz_x'] = np.int64(ucan_md['rows'].str.replace("'","").replace(" ",""))
ucan_md['imgsz_y'] = np.int64(ucan_md['columns'])
ucan_md['num_slices'] = np.int64(ucan_md['num_slices'])
ucan_md['voxsz_x'] = ucan_md['pixel_spacing'].apply(lambda x: (np.float64(x.replace(" ","").replace("[","").replace("]","").split(',')[0])))
ucan_md['voxsz_y'] = ucan_md['pixel_spacing'].apply(lambda x: (np.float64(x.replace(" ","").replace("[","").replace("]","").split(',')[1])))
ucan_md['slice_thickness'] = np.float64(ucan_md['slice_thickness'].str.replace("'","").replace(" ",""))
ucan_md.head(2)

In [ ]:
ucan_md['image_size'] = ucan_md.apply(lambda x: (x.imgsz_x, x.imgsz_y, x.num_slices), axis=1)
ucan_md['voxel_size'] = ucan_md.apply(lambda x: (x.voxsz_x, x.voxsz_y, x.slice_thickness), axis=1)
ucan_md.head(2)

In [ ]:
ucan_md.columns

In [ ]:
#order columns
new_col_lst = ['dir', 'source_dir', 'patient_dir', 'image_dir', 'dicom_img',
       'patient_id', 'patient_age', 'patient_sex', 'patient_weight',
       'patient_size', 'rows', 'columns', 'imgsz_x', 'imgsz_y', 'num_slices', 
       'voxsz_x', 'voxsz_y', 'slice_thickness',  'pixel_spacing',
       'image_size', 'voxel_size', 'img_pos', 'img_orient', 'for_uid', 'att_corr',
       'recons_method', 'image_type', 'aquisition_dt', 'aquisition_time', 'study_desc',
       'series_desc', 'protocol', 'corr_img', 'modality', 'manufacturer',
       'manufacturer_model', 'radiopharmaceutical', 'radiopharmaceutical_volume',
       'radiopharmaceutical_start_time', 'radionuclide_total_dose', 
       'radionuclide_half_life', 'radionuclide_positron_fraction', 
       'radiopharmaceutical_start_date_time'
       ]

ucan_md = ucan_md[new_col_lst]

In [ ]:
print("Toal CT folders: ", ucan_md[ucan_md['image_dir'].str.contains('CT')].shape[0])
print("Toal PT folders: ", ucan_md[ucan_md['image_dir'].str.contains('PT')].shape[0])

In [ ]:
ucan_md.to_excel(destination_path)

In [ ]:
ucan_md.num_slices.min()

In [ ]:
ucan_md[ucan_md['num_slices']<=100].to_excel(os.path.join(source_path, 'exams_with_less_than_100_slices.xlsx'))

lessthan100slices_summ = ucan_md[np.int64(ucan_md['num_slices'])<=100].groupby(['modality', 'slice_thickness', 'num_slices']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
lessthan100slices_summ.to_excel(os.path.join(source_path, 'exams_with_less_than_100_slices_final.xlsx'))

print(lessthan100slices_summ.shape)
#(785, 4)
lessthan100slices_summ.head(10)

In [ ]:
ucan_md_voxeldist = ucan_md.groupby(['modality', 'voxel_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_voxeldist.to_excel(os.path.join(source_path, 'ucan_exams_md_voxeldist.xlsx'))
ucan_md_voxeldist.head(10)


In [ ]:
ucan_md_imgszdist = ucan_md.groupby(['modality', 'image_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_imgszdist.to_excel(os.path.join(source_path, 'ucan_exams_md_imgszdist.xlsx'))
ucan_md_imgszdist.head(10)

In [ ]:
ucan_md_gt200slices = ucan_md[ucan_md['num_slices']>=200]
ucan_md_gt200slices.to_excel(os.path.join(source_path, 'ucan_exams_with_more_than_200_slices.xlsx'))
print(ucan_md_gt200slices.shape)
ucan_md_gt200slices.head(2)

In [ ]:
ucan_md_gt200slices.groupby('voxel_size').agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False)

In [ ]:
ucan_md_gt200slices_voxeldist = ucan_md_gt200slices.groupby(['modality', 'voxel_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_gt200slices_voxeldist.to_excel(os.path.join(source_path, 'ucan_exams_with_more_than_200_slices_voxeldist.xlsx'))
ucan_md_gt200slices_voxeldist.head(10)


In [ ]:
ucan_md_gt200slices_imgszdist = ucan_md_gt200slices.groupby(['modality', 'image_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_gt200slices_imgszdist.to_excel(os.path.join(source_path, 'ucan_exams_with_more_than_200_slices_imagesizedist.xlsx'))
ucan_md_gt200slices_imgszdist.head(10)

In [ ]:
Lessthan_200Slices = ucan_md[ucan_md['num_slices']<200]
Lessthan_200Slices.to_excel(os.path.join(source_path, "less_than_200_slices.xlsx"))
print(Lessthan_200Slices.shape)
Lessthan_200Slices.head(2)

In [ ]:
ucan_md[ucan_md['image_dir'].str.contains('-ax-')][ucan_md['num_slices']<200][ucan_md['image_dir'].str.contains('PT-')]

### Match metadata with selected files

In [21]:
metadata = pd.read_excel(destination_path)
# metadata['dir'] = metadata['dir'].str.replace('/','\\')
print(metadata.shape)
metadata.head(2)

(15224, 44)


,Unnamed: 0,dir,source_dir,patient_dir,image_dir,dicom_img,patient_id,patient_age,patient_sex,patient_weight,patient_size,rows,columns,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,pixel_spacing,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr965491876506-20160627/CT-20160627-103856-6.3_UAS-WB-FDG-3D_CT_VENFAS-CT_SCOUT_HEAD_IN-1250.090942mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr965491876506-20160627,CT-20160627-103856-6.3_UAS-WB-FDG-3D_CT_VENFAS-CT_SCOUT_HEAD_IN-1250.090942mm,1.2.840.113619.2.55.3.4240671514.769.1467005417.24.1,'ASPTCTX0001_npr965491876506','087Y','O','60.0','1.79',2291,888,2291,888,1,0.545455,0.596847,1250.090942,"[0.545455, 0.596847]","(2291, 888, 1)","(0.545455, 0.596847, 1250.090942)","[-265.000, 0.000, 350.000]","[1.000000, -0.000000, 0.000000, 0.000000, 0.000000, -1.000000]",1.2.840.113619.2.55.3.4240671514.769.1467005417.22.14165.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'LOCALIZER']",'20160627','103910','PET FDG WB 1 h','CT SCOUT HEAD IN','6.3 UAS-WB-FDG-3D CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr965491876506-20160627/PT-20160627-104155-UAS-WB-FDG-3D_CT_VENFAS-WB-FDG-AC-3.2700mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr965491876506-20160627,PT-20160627-104155-UAS-WB-FDG-3D_CT_VENFAS-WB-FDG-AC-3.2700mm,1.2.840.113619.2.131.4240671514.1467018651.970210,'ASPTCTX0001_npr965491876506','087Y','O','60.0','1.79',128,128,128,128,247,3.906250,3.906250,3.270000,"[3.90625, 3.90625]","(128, 128, 247)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, 140.250000000000]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.769.1467005417.22.14165.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20160627','105705','PET FDG WB 1 h','WB-FDG-AC','UAS-WB-FDG-3D CT VENFAS',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",'PT','GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','093100.00','242451904.0','6588.0','0.97000002861023','20160627093100.00'


In [22]:
selected_imgs = pd.read_excel(selection_dataframe)
print(selected_imgs.shape)
selected_imgs.head(2)

(3646, 8)


,Unnamed: 0,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date
0,0,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr999612862346-20210628/CT-20210628-114331-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_3_ax-3.000000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr999612862346-20210628,CT-20210628-114331-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_3_ax-3.000000mm,ASPTCTX0001,npr999612862346,20210628
1,1,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr999612862346-20210628/PT-20210628-112746-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-QCFX-S_400_M.Free-2.8000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr999612862346-20210628,PT-20210628-112746-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-QCFX-S_400_M.Free-2.8000mm,ASPTCTX0001,npr999612862346,20210628


In [23]:
selected_imgs.patient_directory.nunique()

1823

In [24]:
master_data = pd.merge(selected_imgs, metadata, how="inner", left_on=['patient_directory', 'PET-CT_info'], right_on=['patient_dir', 'image_dir'], sort=True, suffixes=("_x", "_y"))

col_drop_lst = ['dir', 'source_dir', 'patient_dir',	'image_dir', 'patient_sex', 'rows',	'columns', 'pixel_spacing']

master_data = master_data.drop(columns=col_drop_lst)

master_data['patient_id'] = master_data['patient_id'].apply(lambda x: x.split('_')[1].replace("'",""))
master_data['patient_age'] = master_data['patient_age'].apply(lambda x: x.replace("'","").replace("Y","").replace(" ",""))#.astype('Int64')
master_data['modality'] = master_data['modality'].apply(lambda x: x.strip().replace("'","").strip())

master_data.to_excel(os.path.join(source_path, "Excel_files/06_11_2023/Finalized_dataset.xlsx"))

print(master_data.shape)
master_data.head(2)

(3646, 44)


,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,3689,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,ASPTCTX0001,lpr385705046400,20140313,13743,1.2.840.113619.2.55.3.4240671514.332.1394175950.81.1,lpr385705046400,033,'69.0','1.73',512,512,215,0.976562,0.976562,3.75,"(512, 512, 215)","(0.976562, 0.976562, 3.75)","[-250.000, -250.000, 167.080]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20140313','152608','FDG WB 1 h','STANDARD CT RECON','6.1 UAS-WB-FDG-3D',NaN,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3690,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,ASPTCTX0001,lpr385705046400,20140313,13745,1.2.840.113619.2.131.4240671514.1394722651.463822,lpr385705046400,033,'69.0','1.73',128,128,247,3.906250,3.906250,3.27,"(128, 128, 247)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, 169.020004272460]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20140313','154238','FDG WB 1 h','UAS-WB-FDG-3D-AC','UAS-WB-FDG-3D',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'


In [25]:
print(master_data[master_data['patient_age']==''].shape)

master_data[master_data['patient_age']==''].head(2)

(17, 44)


,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
371,3315,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr205346582030-20160608/PT-20160608-103811-WB_F18_3D_3MIN-AC_3D_50CM-3.2700mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr205346582030-20160608,PT-20160608-103811-WB_F18_3D_3MIN-AC_3D_50CM-3.2700mm,ASPTCTX0001,npr205346582030,20160608,12494,1.2.840.113619.2.131.2831166465.1465376476.374065,npr205346582030,,'92.0','1.85',128,128,289,3.906250,3.906250,3.27,"(128, 128, 289)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, 140.479995727539]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.2831166465.61.1465279152.607.2602.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20160608','105629','PET FDG WB','AC 3D 50CM','WB F18 3D 3MIN',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery ST','FDG -- fluorodeoxyglucose','0.0','090800.00','464979136.0','6588.0','0.97000002861023','20160608090800.00'
445,3241,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr216618605686-20180414/PT-20180414-101021-6.39_WB_FDG_CT_VENFAS-WB_AC_FDG_VPFX-S-2.7900mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr216618605686-20180414,PT-20180414-101021-6.39_WB_FDG_CT_VENFAS-WB_AC_FDG_VPFX-S-2.7900mm,ASPTCTX0001,npr216618605686,20180414,13276,1.2.840.113619.2.405.2831173121.1523694125.517831,npr216618605686,,'75.0','1.71',256,256,287,1.953125,1.953125,2.79,"(256, 256, 287)","(1.953125, 1.953125, 2.79)","[-249.02343750000, -249.02343750000, 129.000000000000]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.405.3.2831173121.966.1523598531.396.9185.1,"'measured,, 0.000000 cm-1, attenuation smooth'",'VPFXS',"['ORIGINAL', 'PRIMARY']",'20180414','101828','PET FDG WB 1 h','WB AC FDG VPFX-S','6.39 WB_FDG CT VENFAS',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RANSNG', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery MI','FDG -- fluorodeoxyglucose','0.0','091200.00','217558320.0','6586.2001953125','0.96700000762939','20180414091200.00'


In [27]:
master_data_imgszdist = master_data.groupby(['modality', 'image_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
master_data_imgszdist.to_excel(os.path.join(source_path, "Excel_files/06_11_2023/master_data_imgszdist.xlsx"))
print(master_data_imgszdist.shape)
master_data_imgszdist.head(10)

(210, 3)


,modality,image_size,dicom_img
0,PT,"(128, 128, 247)",387
1,PT,"(256, 256, 341)",264
2,PT,"(256, 256, 314)",263
3,PT,"(256, 256, 287)",250
4,CT,"(512, 512, 382)",244
5,CT,"(512, 512, 354)",241
6,CT,"(512, 512, 322)",224
7,PT,"(128, 128, 287)",213
8,CT,"(512, 512, 215)",177
9,CT,"(512, 512, 1151)",164


In [28]:
master_data_voxszdist = master_data.groupby(['modality', 'voxel_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
master_data_voxszdist.to_excel(os.path.join(source_path, "Excel_files/06_11_2023/master_data_voxszdist.xlsx"))
print(master_data_voxszdist.shape)
master_data_voxszdist.head(10)

(334, 3)


,modality,voxel_size,dicom_img
0,PT,"(3.90625, 3.90625, 3.27)",722
1,PT,"(1.953125, 1.953125, 2.79)",612
2,CT,"(0.976562, 0.976562, 3.0)",402
3,PT,"(1.953125, 1.953125, 2.8)",394
4,CT,"(0.976562, 0.976562, 3.75)",252
5,CT,"(0.976562, 0.976562, 0.625)",123
6,CT,"(0.976562, 0.976562, 0.949219)",111
7,PT,"(5.46875, 5.46875, 3.27)",88
8,CT,"(1.367188, 1.367188, 3.75)",56
9,CT,"(0.998245, 0.998245, 3.0)",28


In [ ]:
master_data.columns

In [44]:
master_data = pd.read_excel(os.path.join(source_path, "Excel_files/06_11_2023/Finalized_dataset.xlsx"))

#master_data['modality'] = master_data['modality'].apply(lambda x: x.strip().replace("'","").strip())
#master_data.to_csv('F:/U-CAN-Lymfom_A/Selected_for_UCAN_project/Selected_exams_from_U-CAN-Lymfom_A_MasterDataset_2Oct2023.csv', index=False)


### Check no of patients having age in the metadata

In [46]:
print("Master data shape: ", master_data.shape)
print("Total number of patients: ", master_data.npr.nunique())
print("Total number of exams: ", master_data.patient_directory.nunique())
master_data.head(2)

Master data shape:  (3646, 45)
Total number of patients:  1076
Total number of exams:  1823


,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,3689,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,ASPTCTX0001,lpr385705046400,20140313,13743,1.2.840.113619.2.55.3.4240671514.332.1394175950.81.1,lpr385705046400,33.0,'69.0','1.73',512,512,215,0.976562,0.976562,3.75,"(512, 512, 215)","(0.976562, 0.976562, 3.75)","[-250.000, -250.000, 167.080]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20140313','152608','FDG WB 1 h','STANDARD CT RECON','6.1 UAS-WB-FDG-3D',NaN,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3690,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,ASPTCTX0001,lpr385705046400,20140313,13745,1.2.840.113619.2.131.4240671514.1394722651.463822,lpr385705046400,33.0,'69.0','1.73',128,128,247,3.906250,3.906250,3.27,"(128, 128, 247)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, 169.020004272460]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20140313','154238','FDG WB 1 h','UAS-WB-FDG-3D-AC','UAS-WB-FDG-3D',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'


In [48]:
master_data.patient_age.unique()

array([33., 32., 69., 68., 77., 79., 74., 54., 55., 62., 76., 48., 78.,
       81., 75., 80., 71., 72., 82., 58., 59., 41., 52., 65., 67., 27.,
       28., 70., 63., 86., 30., 31., 57., 53., 84., 85., 46., 56., 19.,
       20., 21., 66., 47., 40., 64., 73., 49., 61.,  0., nan, 50., 35.,
       12., 15., 16., 18., 22., 23., 43., 44., 38., 42., 83., 37., 39.,
       60., 36., 24., 29., 45., 51., 34., 26., 25., 89., 87.])

In [49]:
master_data_having_age = master_data[master_data.patient_age>0].copy()
print("Master data shape: ", master_data_having_age.shape)
print("Total number of patients: ", master_data_having_age.npr.nunique())
print("Total number of exams: ", master_data_having_age.patient_directory.nunique())
master_data_having_age.head(2)

Master data shape:  (3610, 45)
Total number of patients:  1070
Total number of exams:  1805


,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,3689,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,ASPTCTX0001,lpr385705046400,20140313,13743,1.2.840.113619.2.55.3.4240671514.332.1394175950.81.1,lpr385705046400,33.0,'69.0','1.73',512,512,215,0.976562,0.976562,3.75,"(512, 512, 215)","(0.976562, 0.976562, 3.75)","[-250.000, -250.000, 167.080]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20140313','152608','FDG WB 1 h','STANDARD CT RECON','6.1 UAS-WB-FDG-3D',NaN,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3690,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,ASPTCTX0001,lpr385705046400,20140313,13745,1.2.840.113619.2.131.4240671514.1394722651.463822,lpr385705046400,33.0,'69.0','1.73',128,128,247,3.906250,3.906250,3.27,"(128, 128, 247)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, 169.020004272460]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20140313','154238','FDG WB 1 h','UAS-WB-FDG-3D-AC','UAS-WB-FDG-3D',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'


In [50]:
master_data_having_age.to_excel(os.path.join(source_path, "Excel_files/06_11_2023/Finalized_dataset_1805_exams_with_Age.xlsx"))

##### Summary for CT images

In [30]:
print("Mode of x image dims: ", master_data[master_data['modality']=='CT']['imgsz_x'].mode()[0])
print("Mode of y image dims: ", master_data[master_data['modality']=='CT']['imgsz_y'].mode()[0])
print("Mode of z image dims: ", master_data[master_data['modality']=='CT']['num_slices'].mode()[0])
print("Mode of x voxel dims: ", master_data[master_data['modality']=='CT']['voxsz_x'].mode()[0])
print("Mode of y voxel dims: ", master_data[master_data['modality']=='CT']['voxsz_y'].mode()[0])
print("Mode of z voxel dims: ", master_data[master_data['modality']=='CT']['slice_thickness'].mode()[0])

Mode of x image dims:  512
Mode of y image dims:  512
Mode of z image dims:  382
Mode of x voxel dims:  0.976562
Mode of y voxel dims:  0.976562
Mode of z voxel dims:  3.0


In [31]:
master_data[master_data['modality']=='CT'][['imgsz_x','imgsz_y','num_slices','voxsz_x','voxsz_y','slice_thickness']].describe()

,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness
count,1823.0,1823.0,1823.000000,1823.000000,1823.000000,1823.000000
mean,512.0,512.0,605.657159,0.953343,0.953343,2.466594
std,0.0,0.0,441.589569,0.109606,0.109606,1.111460
min,512.0,512.0,180.000000,0.703125,0.703125,0.625000
25%,512.0,512.0,322.000000,0.900391,0.900391,0.949219
50%,512.0,512.0,361.000000,0.976562,0.976562,3.000000
75%,512.0,512.0,1151.000000,0.976562,0.976562,3.000000
max,512.0,512.0,1810.000000,1.826172,1.826172,3.750000


##### Summary of PET images

In [32]:
print("Mode of x image dims: ", master_data[master_data['modality']=='PT']['imgsz_x'].mode()[0])
print("Mode of y image dims: ", master_data[master_data['modality']=='PT']['imgsz_y'].mode()[0])
print("Mode of z image dims: ", master_data[master_data['modality']=='PT']['num_slices'].mode()[0])
print("Mode of x voxel dims: ", master_data[master_data['modality']=='PT']['voxsz_x'].mode()[0])
print("Mode of y voxel dims: ", master_data[master_data['modality']=='PT']['voxsz_y'].mode()[0])
print("Mode of z voxel dims: ", master_data[master_data['modality']=='PT']['slice_thickness'].mode()[0])

Mode of x image dims:  256
Mode of y image dims:  256
Mode of z image dims:  287
Mode of x voxel dims:  1.953125
Mode of y voxel dims:  1.953125
Mode of z voxel dims:  3.27


In [33]:
master_data[master_data['modality']=='PT'][['imgsz_x','imgsz_y','num_slices','voxsz_x','voxsz_y','slice_thickness']].describe()

,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness
count,1823.000000,1823.000000,1823.000000,1823.000000,1823.000000,1823.000000
mean,199.126714,199.126714,292.613275,2.898082,2.898082,3.006484
std,63.619416,63.619416,39.037419,1.102951,1.102951,0.236736
min,128.000000,128.000000,207.000000,1.953125,1.953125,2.790000
25%,128.000000,128.000000,249.000000,1.953125,1.953125,2.790000
50%,256.000000,256.000000,287.000000,1.953125,1.953125,2.800000
75%,256.000000,256.000000,314.000000,3.906250,3.906250,3.270000
max,256.000000,256.000000,503.000000,5.468750,5.468750,3.270000


### New size and spacing based on master data

new_size = [384, 384, 384]
new_spacing = [2, 2, 3]

In [35]:
master_data.npr.nunique()

1076

In [34]:
master_data.patient_directory.nunique()

1823